In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -U openai==0.28.0 datasets transformers peft fire sentencepiece sacrebleu bitsandbytes --quiet

In [3]:
import openai
import time
import json
import re

In [5]:
model_name = "gpt-3.5-turbo-instruct"
finetuned_model = "NghiemAbe/flan-t5-base-mathqa_v3"
device = "cuda"
data_path = "./data/test/all_test_round1.json"
OPEN_API_KEY = [
    'sk-flmnoDOca...NgtMxG4Y5kyxsj',
    'sk-tfHXOfLqs...vZPyciJn3OVT0j',
    'sk-h1s7Mc2WM...sfkhPnzcXwx2bv',
    'sk-3tsK40CBq...faxLf18MXrAD2s',
    'sk-Je0HwHPXb...5vDPrDvzYlM1o2',
    'sk-JHNYqDSfQ...aN6VwQIUO65fbx',
    'sk-NBfYfJsQM...OfqIdHhdpUypqe',
    'sk-xEKRqAPtS...t9QcaAUV1OhG0N',
    'sk-tiSIsPpdU...ASByPrfRtQasrZ',
    'sk-AvS3nk3VE...rKxiZMfQ0zhT7Q',
    'sk-jpqVKfQ8i...JCUZvLhT8NRlhR',
    # 'sk-',    # As much as possible :)
]


In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained(finetuned_model)
model = T5ForConditionalGeneration.from_pretrained(finetuned_model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [8]:
def read_jsonl(path: str):
    with open(path) as f:
        data = json.load(f)
        return data


def get_data(path):
    '''
        If there are many json files to load:
        path_item = os.path.join("data/", f"{path}.jsonl")
        data_item = read_jsonl(path_item)
    '''
    data = read_jsonl(path)
    print(f"Loaded {len(data)} examples")
    return data

In [9]:
def get_response(user_request, max_len, temp):
    """
        Get a response from the OpenAI API based on the given arguments.

        Args:
            args (object): The arguments object containing the model name, user request, maximum length, and temperature.
            user_request (str): The user's request to be used as the prompt for generating a response.
            max_len (int): The maximum number of tokens in the generated response.
            temp (float): The temperature parameter for controlling the randomness of the generated response.

        Returns:
            object: The response generated by the OpenAI API.
    """
    responese = openai.Completion.create(
        engine = model_name,
        prompt = user_request,
        max_tokens = max_len,
        n = 1,
        temperature = temp
    )

    return responese

def get_multiple_choice_answer(prompt):
    tokens = tokenizer([prompt], padding=False, return_tensors="pt").to(device)
    out = model.generate(
        **tokens, max_length=10, pad_token_id=model.config.eos_token_id
    )
    result = tokenizer.batch_decode(out, skip_special_tokens=True)[0]
    return result

def convert_to_submit_file(api_result: list = []):
    if api_result[0].isalpha():
        return api_result[0].lower()
    else:
        if 'frac' in api_result or 'pi' in api_result:
            api_result = api_result.split()[0].replace('$','')
        else:
            api_result = ''.join(re.findall(r'[0-9\.\-]+', api_result))
        if len(api_result) and api_result[-1] == '.':
            api_result = api_result[:-1]
        answer = api_result
        return answer.lower()
    return 'Nan'

def run():
    api_keys = OPEN_API_KEY
    key_index = 0
    test_examples = get_data(data_path)
    results = []
    with open('./results/result.txt', 'r') as read:
        results = read.readlines()
    curr_indx = 1
    last_indx = len(results)
    print("Last request: ", last_indx)
    max_len = 10
    temp = 0.05
    prompt = "Please give me a number that is the answer to this question, no explanation, no units, just a single number:"

    with open('./results/result.txt', 'a+') as f:
        for problem in test_examples:
            ques = problem["Problem"]
            options = problem["options"]
            responese = {}
            if curr_indx > last_indx:
                if options != '':
                    start = time.time()
                    result = get_multiple_choice_answer("Give me the answer of this question:" + ques + '\n'+ options)
                    finish = time.time()
                    time_request = finish - start
                    print(f"Time generate for {problem['id']}: {time_request}, answer: {result}")
                else:
                    while 'id' not in responese:
                        try:
                            openai.api_key = api_keys[key_index]
                            print("API KEY: ", openai.api_key[:10] + "..." + openai.api_key[-10:])
                            start = time.time()
                            user_request = prompt + ques
                            responese = get_response(user_request, max_len, temp)
                            finish = time.time()
                            time_request = finish - start
                            result = responese.choices[0].text.strip()
                        except openai.error.RateLimitError:
                            if key_index == 0:
                                print("Waiting...")
                                time.sleep(5)
                            continue
                        finally:
                            key_index = (key_index + 1) % len(api_keys)
                    print(f"Time request for {problem['id']}: {time_request}, answer: {result}")
                choose = convert_to_submit_file(result)
                f.write(choose + '\t' + str(time_request) + '\n')
            curr_indx += 1

In [ ]:
run()